In [ ]:
#-*- coding=utf-8 -*-
import csv
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
 
f = open('kbreport_result.csv','w',encoding='cp949',newline='')
writer = csv.writer(f)
writer.writerow(['선수코드','이름','생년월일'])
 
emptyCount = 0
 
#이름에 붙은 특수문자 및 괄호 제거
def convert(string):
    clean1 = string.replace('\t','').replace('\r','').replace('\n',' ').replace(' ','')
    p = re.compile(r'\(.*?\)')
    clean2 = re.sub(r'[=+,#/\?:^$.@*\"※~&%ㆍ!』\\'|\[\]\<\>`\'...》]', '', clean1)
    clean3 = re.sub(p, "", clean2)
    return clean3
 
#프로필
def kbreport(i):
    global emptyCount
    html = requests.get('http://www.kbreport.com/player/detail/' + str(i)).text
    soup = BeautifulSoup(html,'lxml')
 
    spans = soup.select('div[class="player-info-box"] > span')
 
    if len(spans) == 0:
        emptyCount += 1
    else:
        personId = i
        name = convert(spans[0].get_text(strip=True))
        birthdate = convert(spans[1].get_text(strip=True).replace('생년월일',''))
        row = [personId,name,birthdate]
        emptyCount = 0
        writer.writerow(row)
         
#빈 문서가 10번 이상 지속되면 종료
for i in range(1,1700):
    kbreport(i)
    if emptyCount > 10:
        break
 
f.close()
  
today = datetime.today()
todaydate = today.strftime("%Y-%m-%d")
 
url = 'http://www.kbreport.com/leader/standardList/ajax?rows=200&amp;order=oWAR&amp;orderType=DESC&amp;teamId=&amp;defense_no=&amp;year_from=' + str(today.year) + '&year_to=' + str(today.year) + '&gameType=&split01=day&split02_1=' + todaydate + '&split02_2=' + todaydate + '&r_tpa_count=&tpa_count=0&page=1'
 
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table')
thead = soup.find('tr')
column_names = [th.text.strip() for th in thead.find_all('th')]
 
data = []
for row in table.find_all('tr'):
    row_data = []
    for td in row.find_all('td'):
        td_check = str(td).find('data-playerid')
        if td_check != -1:
            link = td.a['href']
            row_data.append(link.split('/')[3])
        else:
            not_link = ''.join(td.stripped_strings)
            if not_link == '':
                 not_link = None
            row_data.append(not_link)
    data.append(row_data)
df = pd.DataFrame(data[1:], columns=column_names)
df = df.drop(['#','팀명','경기','타석','타수','안타','득점','타점','고4','삼진','희플','희타','병살','도실','HBP','타율'],axis=1)
df.rename(columns={'선수명':'personId','단타':'1B','2루타':'2B','3루타':'3B','홈런':'HR','볼넷':'BB','도루':'SB'}, inplace = True)
 
def convert(a):
    if int(a) > 0:
        return 1
    else:
        return 0
 
df.loc[:, '1B':'SB'] = df.loc[:, '1B':'SB'].applymap(convert)
df.to_csv('kbreport' + str(today.strftime("%Y%m%d")) + '.csv',index=False)

import pymysql
from openpyxl import Workbook
  
# MySQL Connection 연결
conn = pymysql.connect(
host='kbomanager.aceproject.co.kr',
port=13306,
user='mplayball',
password='비밀번호',
database='mplayball',
charset='utf8',
cursorclass=pymysql.cursors.DictCursor
)
 
# Connection 으로부터 Cursor 생성
cur = conn.cursor()
  
# 쿼리문 작성
cur.execute("select date(a.crt_date), a.person_id, b.name, a.hit_1b_yn, a.hit_2b_yn, a.hit_3b_yn, a.hr_yn, a.bb_yn, a.sb_yn  from mplayball.live_prp_result a, mplayball.live_person b where a.person_id = b.person_id and date(a.crt_date) = date(date_sub(now(), interval 1 day));")
conn.commit()
result = cur.fetchall()
  
wb = Workbook()
ws = wb.active
wb.remove_sheet(ws)
list = wb.create_sheet("Sales")
  
Dic = result[0]
  
rows=[]
 
for a in Dic.keys():
    rows.append(a)
 
list.append(rows)
 
for i in result:
    rows=[]
    for a in i.values():
        rows.append(a)
    list.append(rows)
 
wb.save('C:\\Users\\JJS\\Desktop\\DB.xlsx')